In [2]:
import os
import pandas as pd
import json
import pprint
import re
import nbformat

from collections import defaultdict

In [3]:
with open("../DataProcessing/TestData/gameData.json", "r", encoding="utf-8") as file:
    data = json.load(file)
    
with open("../DataProcessing/TestData/postGame.json", "r", encoding="utf-8") as file:
    postData = json.load(file)
    
with open("../DataProcessing/TestData/runeSheet.json", "r", encoding="utf-8") as file:
    runeSheet = json.load(file)
    
# Access the 'participants' array within 'metadata'
participantsData = data['metadata']['participants']

# Access the 'frames' array within 'metadata'
framesData = data['info']['frames']

participantFrames = [frame['participantFrames'] for frame in framesData]


# Create dictionaries to store the counts and most mentioned names for each ID
championCountsDict = defaultdict(lambda: defaultdict(int))
namesDict = defaultdict(list)

mostMentionedNamesDict = {}
participantData = []
participantChampionMapping = {}


In [4]:
targetPuuid = '7hSyI8OKEFPpadVyhqKmTOJlT6umda2FMt1sAeIcZ1WcxdATw7Tvu4ixaR8llqv9u2qPutnAsL9Shg'

In [5]:
# Function to calculate champion counts
def calculateChampionCounts(data):
    championCountsDict = {}
    namesDict = {}

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))

                    if killerId not in championCountsDict:
                        championCountsDict[killerId] = {}
                        namesDict[killerId] = []

                    if victimId not in championCountsDict:
                        championCountsDict[victimId] = {}
                        namesDict[victimId] = []

                    # Find the killer's and victim's participant data
                    killerNames = [victim.get('name') for victim in event.get('victimDamageReceived', [])]
                    victimNames = [killer.get('name') for killer in event.get('victimDamageDealt', [])]

                    # Use the last name mentioned as the killer name
                    if killerNames:
                        killerName = killerNames[-1]
                        if killerName not in championCountsDict[killerId]:
                            championCountsDict[killerId][killerName] = 1
                        else:
                            championCountsDict[killerId][killerName] += 1
                        namesDict[killerId].append(killerName)

                    # Use the last name mentioned as the victim name
                    if victimNames:
                        victimName = victimNames[-1]
                        if victimName not in championCountsDict[victimId]:
                            championCountsDict[victimId][victimName] = 1
                        else:
                            championCountsDict[victimId][victimName] += 1
                        namesDict[victimId].append(victimName)

    return championCountsDict, namesDict

def calculateMostMentionedChampionNames(data):
    # Calculate champion counts
    championCountsDict, namesDict = calculateChampionCounts(data)

    # Process the counts and choose the most mentioned champion name for each ID
    mostMentionedNamesDict = {}
    for participantId, championCounts in championCountsDict.items():
        mostMentionedChampion = max(championCounts, key=championCounts.get)
        mostMentionedNamesDict[participantId] = mostMentionedChampion
        count = championCounts[mostMentionedChampion]
        names = namesDict[participantId]

    return mostMentionedNamesDict

mostMentionedNamesDict = calculateMostMentionedChampionNames(data)

In [6]:
def addParticipantData(participantId, puuid, championName):  # add championName
    participantData.append([participantId, puuid, championName])  # add championName

def findParticipantData(data):
    if 'info' in data and 'participants' in data['info']:
        for participant in data['info']['participants']:
            participantId = participant['participantId']
            puuid = participant['puuid']
            
            championName = mostMentionedNamesDict[participantId]
            addParticipantData(participantId, puuid, championName)

findParticipantData(data)

In [7]:
def findParticipantByPuuid(participantData, targetPuuid):
    for entry in participantData:
        participantId, puuid, championName = entry
        if puuid == targetPuuid:
            return participantId, championName
    return None, None

# Example usage:
# Assuming 'targetPuuid' is the Puuid you want to find
targetParticipantId, targetChampionName = findParticipantByPuuid(participantData, targetPuuid)

if targetParticipantId is not None:
    print(f"Participant ID: {targetParticipantId}, Champion: {targetChampionName}")
else:
    print("Participant not found.")

Participant ID: 2, Champion: MasterYi


In [8]:
def excludeColumns(data, columnsToExclude=None, displayNoZero=False):
    # Exclude specified columns
    excludedData = {}

    for key, value in data.items():
        if key in columnsToExclude:
            continue  # Skip excluded columns
        elif isinstance(value, dict):
            # Recursively exclude columns within nested dictionaries
            excludedData[key] = excludeColumns(value, columnsToExclude, displayNoZero)
        else:
            excludedData[key] = value

    return excludedData

def displayNonZeroParams(participantFrame, displayNoZero=False, indent=''):
    # Iterate through the participant frame's items
    for key, value in participantFrame.items():
        if isinstance(value, dict):
            # If the value is a nested dictionary, iterate through its items
            print(f"{indent}{key} for {targetChampionName}: ")
            displayNonZeroParams(value, displayNoZero, indent + '  ')
        elif isinstance(value, (int, float)) and (not displayNoZero or value != 0):
            # Display the parameter if it's a numeric value and not zero
            print(f"{indent}  {key}: {value}")

In [9]:
def getParticipantDataAtMinute(data, participantId, targetMinute, columnsToExclude=None, displayNoZero=False):
    timestampData = []

    # Check if there are frames in the data
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            timestamp = frame['timestamp']
            currentMinute = round(timestamp / 60000)  # Convert milliseconds to minutes and round to the nearest minute

            if currentMinute == targetMinute:
                participantFrames = frame.get('participantFrames', {})
                if participantId in participantFrames:
                    participantData = participantFrames[participantId]
                    timestampData.append([timestamp, participantData])

                    # Call the excludeColumns method
                    excludedData = excludeColumns(participantData, columnsToExclude, displayNoZero)

                    # Display either the original data or the excluded data based on your requirement
                    if displayNoZero:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(excludedData, displayNoZero)
                        print("\n---\n")
                    else:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(participantData, displayNoZero)
                        print("\n---\n")
    return timestampData

desiredParticipantId = (f"{targetParticipantId}")
desiredMinute = 2 
columnsToExclude = ['magicDamageDone', 'magicDamageDoneToChampions', 'magicDamageTaken', 
                    'physicalDamageDone', 'physicalDamageDoneToChampions', 'physicalDamageTaken', 
                    'trueDamageDone', 'trueDamageDoneToChampions', 'trueDamageTaken', 'timeEnemySpentControlled']

# Call the function with displayNoZero set to True
result = getParticipantDataAtMinute(data, desiredParticipantId, desiredMinute, columnsToExclude=columnsToExclude, displayNoZero=True)


Timestamp at minute 2: 120029
championStats for MasterYi: 
    armor: 36
    attackDamage: 71
    attackSpeed: 114
    health: 417
    healthMax: 761
    healthRegen: 23
    magicResist: 33
    movementSpeed: 355
    power: 236
    powerMax: 281
    powerRegen: 34
  currentGold: 161
damageStats for MasterYi: 
    totalDamageDone: 3255
    totalDamageDoneToChampions: 145
    totalDamageTaken: 755
  jungleMinionsKilled: 4
  level: 2
  participantId: 2
position for MasterYi: 
    x: 8578
    y: 2559
  totalGold: 611
  xp: 325

---



In [10]:
def getKillDataForParticipant(data, participantId, participantData):
    killData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))
                    assistIds = [int(assistId) for assistId in event.get('assistingParticipantIds', [])]

                    if killerId == participantId:
                        timestamp = event['timestamp']
                        victimDamageReceived = event.get('victimDamageReceived', [])
                        killPosition = event.get('position', {})  # Added line

                        # Calculate total damage done by the killer to the victim
                        totalDamageDone = sum(
                            damageInfo['basic'] + damageInfo['magicDamage'] + damageInfo['physicalDamage'] + damageInfo['trueDamage']
                            for damageInfo in victimDamageReceived
                            if damageInfo['participantId'] == participantId
                        )

                        # Find names from participantData
                        killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                           f"Unknown Champion with ID {killerId}")
                        victimName = next((participant[2] for participant in participantData if participant[0] == victimId),
                                           f"Unknown Champion with ID {victimId}")

                        # Find names of assisting champions and their damage
                        assistingChampions = []
                        damageByAssists = []

                        for assistId in assistIds:
                            assistName = next((participant[2] for participant in participantData if participant[0] == assistId),
                                               f"Unknown Champion with ID {assistId}")

                            assistDamage = sum(
                                damageInfo['basic'] + damageInfo['magicDamage'] + damageInfo['physicalDamage'] + damageInfo['trueDamage']
                                for damageInfo in victimDamageReceived
                                if damageInfo['participantId'] == assistId
                            )

                            assistingChampions.append(assistName)
                            damageByAssists.append(assistDamage)

                        killData.append({
                            'timestamp': timestamp,
                            'killerId': killerId,
                            'killerName': killerName,
                            'victimId': victimId,
                            'victimName': victimName,
                            'totalDamageDone': totalDamageDone,
                            'assistingChampions': assistingChampions,
                            'damageByAssists': damageByAssists,
                            'position': killPosition  # Added line
                        })

    return killData



killDataForParticipant = getKillDataForParticipant(data, targetParticipantId, participantData)

# Display the kill data with total damage done, assisting champions, and damage by assists
for kill in killDataForParticipant:
    assistingChampionsLine = f"Assisting Champions: {', '.join(kill['assistingChampions'])}.\n" if kill['assistingChampions'] else ""
    damageByAssistsLine = f"Damage Done by Assists: {', '.join([f'{damage} by {assist}' for damage, assist in zip(kill['damageByAssists'], kill['assistingChampions'])])}.\n" if kill['assistingChampions'] else ""
    
    killLocation = f"X: {kill['position'].get('x', 0)}, Y: {kill['position'].get('y', 0)}"
    
    print(f"Timestamp: {kill['timestamp']}\n"
          f"Killer: {kill['killerName']} (ID: {kill['killerId']})\n"
          f"Victim: {kill['victimName']} (ID: {kill['victimId']})\n"
          f"Total Damage Done: {kill['totalDamageDone']}\n"
          f"{assistingChampionsLine}"
          f"{damageByAssistsLine}"
          f"Kill Location: ({killLocation})\n")


Timestamp: 384449
Killer: MasterYi (ID: 2)
Victim: Khazix (ID: 7)
Total Damage Done: 998
Kill Location: (X: 9400, Y: 6528)

Timestamp: 1076595
Killer: MasterYi (ID: 2)
Victim: Shen (ID: 6)
Total Damage Done: 419
Assisting Champions: Jax, Veigar.
Damage Done by Assists: 702 by Jax, 1168 by Veigar.
Kill Location: (X: 4455, Y: 9895)

Timestamp: 1158364
Killer: MasterYi (ID: 2)
Victim: Sylas (ID: 8)
Total Damage Done: 1397
Assisting Champions: Jax, Veigar.
Damage Done by Assists: 723 by Jax, 0 by Veigar.
Kill Location: (X: 4071, Y: 8585)

Timestamp: 1219331
Killer: MasterYi (ID: 2)
Victim: Khazix (ID: 7)
Total Damage Done: 1934
Kill Location: (X: 9128, Y: 1327)

Timestamp: 1316023
Killer: MasterYi (ID: 2)
Victim: Nautilus (ID: 10)
Total Damage Done: 1272
Assisting Champions: Veigar, Varus, Leona.
Damage Done by Assists: 295 by Veigar, 540 by Varus, 69 by Leona.
Kill Location: (X: 8680, Y: 6775)

Timestamp: 1320333
Killer: MasterYi (ID: 2)
Victim: Sylas (ID: 8)
Total Damage Done: 844
Assist

In [11]:
def getAssistsForParticipant(data, targetParticipantId, participantData):
    assistData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    assistIds = [int(assistId) for assistId in event.get('assistingParticipantIds', [])]

                    if targetParticipantId in assistIds:
                        timestamp = event['timestamp']
                        
                        # Get the name and ID of the killer
                        killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                          f"Unknown Champion with ID {killerId}")

                        killerParticipantId = killerId

                        assistData.append({
                            'timestamp': timestamp,
                            'killerId': killerParticipantId,
                            'killerName': killerName,
                            'assistId': targetParticipantId,
                            'assistName': next((participant[2] for participant in participantData if participant[0] == targetParticipantId),
                                               f"Unknown Champion with ID {targetParticipantId}"),
                            'position': event.get('position', {'x': 0, 'y': 0})  # Use default coordinates if not present
                        })

    return assistData

# Example usage: 

assistDataForParticipant = getAssistsForParticipant(data, targetParticipantId, participantData)

# Display the assist data with timestamps, killer information, and coordinates
for assist in assistDataForParticipant:
    
    assistLocation = f"X: {assist['position'].get('x', 0)}, Y: {assist['position'].get('y', 0)}"
    
    print(f"Timestamp: {assist['timestamp']}\n"
          f"Killer: {assist['killerName']} (ID: {assist['killerId']})\n"
          f"Assist: {assist['assistName']} (ID: {assist['assistId']})\n"
          f"Assist Location: ({assistLocation})\n")


Timestamp: 272085
Killer: Jax (ID: 1)
Assist: MasterYi (ID: 2)
Assist Location: (X: 3464, Y: 13074)

Timestamp: 538831
Killer: Leona (ID: 5)
Assist: MasterYi (ID: 2)
Assist Location: (X: 12205, Y: 3217)

Timestamp: 539935
Killer: Leona (ID: 5)
Assist: MasterYi (ID: 2)
Assist Location: (X: 12592, Y: 2977)

Timestamp: 816928
Killer: Varus (ID: 4)
Assist: MasterYi (ID: 2)
Assist Location: (X: 12998, Y: 4356)

Timestamp: 847162
Killer: Varus (ID: 4)
Assist: MasterYi (ID: 2)
Assist Location: (X: 11970, Y: 4447)

Timestamp: 934514
Killer: Varus (ID: 4)
Assist: MasterYi (ID: 2)
Assist Location: (X: 12644, Y: 2564)

Timestamp: 1081605
Killer: Jax (ID: 1)
Assist: MasterYi (ID: 2)
Assist Location: (X: 4765, Y: 9700)

Timestamp: 1086007
Killer: Veigar (ID: 3)
Assist: MasterYi (ID: 2)
Assist Location: (X: 4053, Y: 9372)

Timestamp: 1291441
Killer: Varus (ID: 4)
Assist: MasterYi (ID: 2)
Assist Location: (X: 9537, Y: 5506)

Timestamp: 1292812
Killer: Veigar (ID: 3)
Assist: MasterYi (ID: 2)
Assist Lo

In [12]:
def getDeathsForParticipant(data, targetParticipantId, participantData):
    deathData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    victimId = int(event.get('victimId'))
                    killerId = int(event.get('killerId'))

                    if victimId == targetParticipantId:
                        timestamp = event['timestamp']

                        # Get the name and ID of the killer
                        killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                          f"Unknown Champion with ID {killerId}")

                        killerParticipantId = killerId

                        deathData.append({
                            'timestamp': timestamp,
                            'victimId': targetParticipantId,
                            'victimName': next((participant[2] for participant in participantData if participant[0] == targetParticipantId),
                                               f"Unknown Champion with ID {targetParticipantId}"),
                            'killerId': killerParticipantId,
                            'killerName': killerName,
                            'position': event.get('position', {'x': 0, 'y': 0})  # Use default coordinates if not present
                        })

    return deathData


deathDataForParticipant = getDeathsForParticipant(data, targetParticipantId, participantData)

# Display the death data with timestamps, killer information, and coordinates
for death in deathDataForParticipant:
    
    deathLocation = f"X: {death['position'].get('x', 0)}, Y: {death['position'].get('y', 0)}"
    
    print(f"Timestamp: {death['timestamp']}\n"
          f"Victim: {death['victimName']} (ID: {death['victimId']})\n"
          f"Killer: {death['killerName']} (ID: {death['killerId']})\n"
          f"Death Location: ({deathLocation})\n")


Timestamp: 269056
Victim: MasterYi (ID: 2)
Killer: Khazix (ID: 7)
Death Location: (X: 2200, Y: 12018)

Timestamp: 384416
Victim: MasterYi (ID: 2)
Killer: Khazix (ID: 7)
Death Location: (X: 9477, Y: 6446)

Timestamp: 639690
Victim: MasterYi (ID: 2)
Killer: Shen (ID: 6)
Death Location: (X: 1845, Y: 13567)

Timestamp: 682066
Victim: MasterYi (ID: 2)
Killer: Khazix (ID: 7)
Death Location: (X: 2624, Y: 8467)

Timestamp: 851420
Victim: MasterYi (ID: 2)
Killer: Sylas (ID: 8)
Death Location: (X: 10974, Y: 4618)

Timestamp: 971757
Victim: MasterYi (ID: 2)
Killer: Khazix (ID: 7)
Death Location: (X: 13345, Y: 1686)

Timestamp: 1228359
Victim: MasterYi (ID: 2)
Killer: Shen (ID: 6)
Death Location: (X: 11469, Y: 1644)

Timestamp: 1363679
Victim: MasterYi (ID: 2)
Killer: Khazix (ID: 7)
Death Location: (X: 6266, Y: 12752)

Timestamp: 1514562
Victim: MasterYi (ID: 2)
Killer: Ezreal (ID: 9)
Death Location: (X: 11541, Y: 4509)

Timestamp: 1663734
Victim: MasterYi (ID: 2)
Killer: Shen (ID: 6)
Death Locati

In [13]:
def displayParticipantParameters(data, participantIds):

    # Display the extracted parameter values
    for participantId in participantIds:
        print(f"Participant {targetChampionName} Parameters:")
        for idx, frameData in enumerate(participantFrames):
            damageStats = frameData.get(participantId, {}).get('damageStats', {})
            totalDamageDone = damageStats.get('totalDamageDone', 0)
            totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)
            totalDamageTaken = damageStats.get('totalDamageTaken', 0)

            print(f"Minute {idx + 1}:")
            print(f"  Total Damage Done: {totalDamageDone}")
            print(f"  Total Damage Done To Champions: {totalDamageDoneToChampions}")
            print(f"  Total Damage Taken: {totalDamageTaken}")

# Example usage:
# Assuming 'data' is the loaded JSON data
displayParticipantParameters(data, f"{targetParticipantId}")

Participant MasterYi Parameters:
Minute 1:
  Total Damage Done: 0
  Total Damage Done To Champions: 0
  Total Damage Taken: 0
Minute 2:
  Total Damage Done: 145
  Total Damage Done To Champions: 145
  Total Damage Taken: 0
Minute 3:
  Total Damage Done: 3255
  Total Damage Done To Champions: 145
  Total Damage Taken: 755
Minute 4:
  Total Damage Done: 10871
  Total Damage Done To Champions: 145
  Total Damage Taken: 1725
Minute 5:
  Total Damage Done: 16416
  Total Damage Done To Champions: 145
  Total Damage Taken: 2765
Minute 6:
  Total Damage Done: 17705
  Total Damage Done To Champions: 504
  Total Damage Taken: 3747
Minute 7:
  Total Damage Done: 27259
  Total Damage Done To Champions: 504
  Total Damage Taken: 4768
Minute 8:
  Total Damage Done: 28691
  Total Damage Done To Champions: 1502
  Total Damage Taken: 5764
Minute 9:
  Total Damage Done: 32242
  Total Damage Done To Champions: 1502
  Total Damage Taken: 6129
Minute 10:
  Total Damage Done: 37783
  Total Damage Done To Ch

In [14]:
def calculateDamageDifference(data, targetParticipantId, startFrame, endFrame, participantData):
    # Extract the numeric part of the targetParticipantId
    
    participantId = ''.join(filter(str.isdigit, targetParticipantId))

    # Determine the extra participantId based on the given participantId
    extraParticipantId = str((int(participantId) + 5) % 10) if int(participantId) != 5 else '10'

    # Get the names of the participant and extra participant
    participantName = next((participant[2] for participant in participantData if participant[0] == int(participantId)),
                           f"Unknown Champion with ID {participantId}")

    extraParticipantName = next((participant[2] for participant in participantData if participant[0] == int(extraParticipantId)),
                                f"Unknown Champion with ID {extraParticipantId}")

    # Initialize variables to store total damage done to champions
    totalDamageToChampionsStart = 0
    totalDamageToChampionsEnd = 0
    totalDamageToChampionsExtraStart = 0
    totalDamageToChampionsExtraEnd = 0

    # Display the extracted parameter values and calculate total damage done to champions
    for idx, frameData in enumerate(participantFrames):
        damageStats = frameData.get(participantId, {}).get('damageStats', {})
        totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)

        extraDamageStats = frameData.get(extraParticipantId, {}).get('damageStats', {})
        totalDamageDoneToChampionsExtra = extraDamageStats.get('totalDamageDoneToChampions', 0)

        if idx + 1 == startFrame:
            totalDamageToChampionsStart = totalDamageDoneToChampions
            totalDamageToChampionsExtraStart = totalDamageDoneToChampionsExtra
        elif idx + 1 == endFrame:
            totalDamageToChampionsEnd = totalDamageDoneToChampions
            totalDamageToChampionsExtraEnd = totalDamageDoneToChampionsExtra

    # Print the total damage done to champions for the participant and extra participant
    print(f"PRE")
    print(f"Minute {startFrame} - Participant {participantName}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsStart}")
    print(f"Minute {startFrame} - Extra Participant {extraParticipantName}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsExtraStart}")

    print(f"POST")
    print(f"Minute {endFrame} - Participant {participantName}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsEnd}")
    print(f"Minute {endFrame} - Extra Participant {extraParticipantName}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsExtraEnd}")

    # Calculate and print the differences
    difference = totalDamageToChampionsEnd - totalDamageToChampionsStart
    extraDifference = totalDamageToChampionsExtraEnd - totalDamageToChampionsExtraStart
    print(f"\nTotal Damage Done To Champions Difference (Participant {participantName}): {difference}")
    print(f"Total Damage Done To Champions Difference (Extra Participant {extraParticipantName}): {extraDifference}")

# Example usage: f'{targetParticipantId}'
# Assuming 'data' is the loaded JSON data and 'participantData' is the participant data
calculateDamageDifference(data, f'{targetParticipantId}', 5, 10, participantData)


PRE
Minute 5 - Participant MasterYi:
  Total Damage Done To Champions: 145
Minute 5 - Extra Participant Khazix:
  Total Damage Done To Champions: 0
POST
Minute 10 - Participant MasterYi:
  Total Damage Done To Champions: 2550
Minute 10 - Extra Participant Khazix:
  Total Damage Done To Champions: 4226

Total Damage Done To Champions Difference (Participant MasterYi): 2405
Total Damage Done To Champions Difference (Extra Participant Khazix): 4226


In [15]:
def calculateDamageDifference(data, participantId, startFrame, endFrame):

    # Determine the extra participantId based on the given participantId
    extraParticipantId = str((int(participantId) + 5) % 10) if int(participantId) != 5 else '10'
                                                                                            

    extraParticipantName = next((participant[2] for participant in participantData if participant[0] == int(extraParticipantId)),
                                f"Unknown Champion with ID {extraParticipantId}")
    
    # Initialize variables to store total damage done parameters
    totalDamageDoneStart = 0
    totalDamageDoneEnd = 0
    totalDamageDoneExtraStart = 0
    totalDamageDoneExtraEnd = 0

    totalDamageDoneToChampionsStart = 0
    totalDamageDoneToChampionsEnd = 0
    totalDamageDoneToChampionsExtraStart = 0
    totalDamageDoneToChampionsExtraEnd = 0

    totalDamageTakenStart = 0
    totalDamageTakenEnd = 0
    totalDamageTakenExtraStart = 0
    totalDamageTakenExtraEnd = 0
    
    totalDamageTakenCompared = 0
    totalDamageDoneCompared = 0
    totalDamageDoneToChampionsCompared = 0

    # Display the extracted parameter values and calculate total damage done parameters
    for idx, frameData in enumerate(participantFrames):
        damageStats = frameData.get(participantId, {}).get('damageStats', {})
        totalDamageDone = damageStats.get('totalDamageDone', 0)
        totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)
        totalDamageTaken = damageStats.get('totalDamageTaken', 0)

        extraDamageStats = frameData.get(extraParticipantId, {}).get('damageStats', {})
        totalDamageDoneExtra = extraDamageStats.get('totalDamageDone', 0)
        totalDamageDoneToChampionsExtra = extraDamageStats.get('totalDamageDoneToChampions', 0)
        totalDamageTakenExtra = extraDamageStats.get('totalDamageTaken', 0)

        if idx + 1 == startFrame:
            totalDamageDoneStart = totalDamageDone
            totalDamageDoneToChampionsStart = totalDamageDoneToChampions
            totalDamageTakenStart = totalDamageTaken
            totalDamageDoneExtraStart = totalDamageDoneExtra
            totalDamageDoneToChampionsExtraStart = totalDamageDoneToChampionsExtra
            totalDamageTakenExtraStart = totalDamageTakenExtra
        elif idx + 1 <= endFrame:
            totalDamageDoneEnd = totalDamageDone
            totalDamageDoneToChampionsEnd = totalDamageDoneToChampions
            totalDamageTakenEnd = totalDamageTaken
            totalDamageDoneExtraEnd = totalDamageDoneExtra
            totalDamageDoneToChampionsExtraEnd = totalDamageDoneToChampionsExtra
            totalDamageTakenExtraEnd = totalDamageTakenExtra
    
    # Print the pre and post data for the first participant
    print(f"Participant {targetChampionName}:")
    print(f"  Minute {startFrame} - Total Damage Done: {totalDamageDoneStart}")
    print(f"  Minute {endFrame} - Total Damage Done: {totalDamageDoneEnd}")
    print(f"  Minute {startFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsStart}")
    print(f"  Minute {endFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsEnd}")
    print(f"  Minute {startFrame} - Total Damage Taken: {totalDamageTakenStart}")
    print(f"  Minute {endFrame} - Total Damage Taken: {totalDamageTakenEnd}")

    # Print the data for the extra participant
    print(f"\nEnemy Laner: {extraParticipantName}")
    print(f"  Minute {startFrame} - Total Damage Done: {totalDamageDoneExtraStart}")
    print(f"  Minute {endFrame} - Total Damage Done: {totalDamageDoneExtraEnd}")
    print(f"  Minute {startFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsExtraStart}")
    print(f"  Minute {endFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsExtraEnd}")
    print(f"  Minute {startFrame} - Total Damage Taken: {totalDamageTakenExtraStart}")
    print(f"  Minute {endFrame} - Total Damage Taken: {totalDamageTakenExtraEnd}")

    # Calculate and print the differences
    damageDoneDifference = totalDamageDoneEnd - totalDamageDoneStart
    extraDamageDoneDifference = totalDamageDoneExtraEnd - totalDamageDoneExtraStart
    damageDoneToChampionsDifference = totalDamageDoneToChampionsEnd - totalDamageDoneToChampionsStart
    extraDamageDoneToChampionsDifference = totalDamageDoneToChampionsExtraEnd - totalDamageDoneToChampionsExtraStart
    damageTakenDifference = totalDamageTakenEnd - totalDamageTakenStart
    extraDamageTakenDifference = totalDamageTakenExtraEnd - totalDamageTakenExtraStart

    print(f"\nTotal Damage Done Difference ({targetChampionName}): {damageDoneDifference}")
    print(f"Total Damage Done Difference ({extraParticipantName}): {extraDamageDoneDifference}")
    print(f"Total Damage Done To Champions Difference ({targetChampionName}): {damageDoneToChampionsDifference}")
    print(f"Total Damage Done To Champions Difference ({extraParticipantName}): {extraDamageDoneToChampionsDifference}")
    print(f"Total Damage Taken Difference ({targetChampionName}): {damageTakenDifference}")
    print(f"Total Damage Taken Difference ({extraParticipantName}): {extraDamageTakenDifference}")
    
    # Calculate differences between participant and enemy
    totalDamageTakenCompared = damageTakenDifference - extraDamageTakenDifference
    totalDamageDoneCompared = damageDoneDifference - extraDamageDoneDifference
    totalDamageDoneToChampionsCompared = damageDoneToChampionsDifference - extraDamageDoneToChampionsDifference
    
    if totalDamageTakenCompared >= 0:
        print(f"\nParticipant {targetChampionName} took {totalDamageTakenCompared} more damage than enemy laner, ({extraParticipantName}) in given timeframe")
    else:
        print(f"\nParticipant {targetChampionName} took {abs(totalDamageTakenCompared)} less damage than enemy laner, ({extraParticipantName}) in given timeframe")

    if totalDamageDoneToChampionsCompared >= 0:
        print(f"\nParticipant {targetChampionName} beat enemy laner, ({extraParticipantName}) in given timeframe with total damage done difference of: {totalDamageDoneToChampionsCompared}")
    else:
        print(f"\nParticipant {targetChampionName} was beat by enemy laner, ({extraParticipantName}) in given timeframe with total damage done difference of: {abs(totalDamageDoneToChampionsCompared)}")

    if totalDamageDoneCompared >= 0:
        print(f"\nParticipant {targetChampionName} beat enemy laner, ({extraParticipantName}) in given timeframe with total damage done difference of: {totalDamageDoneCompared}")
    else:
        print(f"\nParticipant {targetChampionName} was beat by enemy laner, ({extraParticipantName}) in given timeframe with total damage done difference of: {abs(totalDamageDoneCompared)}")

# Example usage:
# Assuming 'data' is the loaded JSON data
calculateDamageDifference(data, f'{targetParticipantId}', 15, 25)


Participant MasterYi:
  Minute 15 - Total Damage Done: 74368
  Minute 25 - Total Damage Done: 135757
  Minute 15 - Total Damage Done To Champions: 6335
  Minute 25 - Total Damage Done To Champions: 27057
  Minute 15 - Total Damage Taken: 14138
  Minute 25 - Total Damage Taken: 27730

Enemy Laner: Khazix
  Minute 15 - Total Damage Done: 75820
  Minute 25 - Total Damage Done: 148113
  Minute 15 - Total Damage Done To Champions: 9104
  Minute 25 - Total Damage Done To Champions: 19969
  Minute 15 - Total Damage Taken: 15691
  Minute 25 - Total Damage Taken: 31357

Total Damage Done Difference (MasterYi): 61389
Total Damage Done Difference (Khazix): 72293
Total Damage Done To Champions Difference (MasterYi): 20722
Total Damage Done To Champions Difference (Khazix): 10865
Total Damage Taken Difference (MasterYi): 13592
Total Damage Taken Difference (Khazix): 15666

Participant MasterYi took 2074 less damage than enemy laner, (Khazix) in given timeframe

Participant MasterYi beat enemy laner

In [16]:
def getVisionScoreAdvantageLaneOpponent(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            challengesData = participant.get('challenges', {})
            return challengesData.get('visionScoreAdvantageLaneOpponent', None)
    return None

def getVisionScore(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('visionScore', None)
    return None

def getKillParticipation(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('killParticipation', None)
    return None

def getGoldPerMinute(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('goldPerMinute', None)
    return None

def getTotalDamageDealtToChampions(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('totalDamageDealtToChampions', None)
    return None

def getTotalDamageTaken(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('totalDamageTaken', None)
    return None

def getControlWardTimeCoverageInRiverOrEnemyHalf(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('controlWardTimeCoverageInRiverOrEnemyHalf', None)
    return None

def getControlWardsPlaced(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('controlWardsPlaced', None)
    return None

def getSoloKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('soloKills', None)
    return None

def getStealthWardsPlaced(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('stealthWardsPlaced', None)
    return None

def getWardsKilled(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('wardsKilled', None)
    return None

def getWardsPlaced(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('wardsPlaced', None)
    return None

def getTeamBaronKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('teamBaronKills', None)
    return None

def getTeamRiftHeraldKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('teamRiftHeraldKills', None)
    return None

def getTeamElderDragonKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('teamElderDragonKills', None)
    return None

def getDragonKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('dragonKills', None)
    return None

def getDeathsByEnemyChamps(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('deathsByEnemyChamps', None)
    return None

def getDragonTakedowns(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('dragonTakedowns', None)
    return None

def getKda(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('kda', None)
    return None

def getTakedowns(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('takedowns', None)
    return None

def getTurretTakedowns(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('turretTakedowns', None)
    return None

def getLaneMinionsFirst10Minutes(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('laneMinionsFirst10Minutes', None)
    return None

def getLaneRole(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('individualPosition', None)
    return None
        
def getJungleStats(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid and participant.get('individualPosition') == 'JUNGLE':
            totalAllyJungleMinionsKilled = participant.get('totalAllyJungleMinionsKilled', None)
            totalEnemyJungleMinionsKilled = participant.get('totalEnemyJungleMinionsKilled', None)
            return totalAllyJungleMinionsKilled, totalEnemyJungleMinionsKilled
    return None, None

def getTeamSide(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('teamId', None)
    return None

def getGameWon(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('win', None)
    return None

laneRole = getLaneRole(postData, targetPuuid)
print(f"Role of {targetChampionName}: {laneRole}")

totalAllyJungleMinionsKilled, totalEnemyJungleMinionsKilled = getJungleStats(postData, targetPuuid)
if totalAllyJungleMinionsKilled is not None and totalEnemyJungleMinionsKilled is not None:
    print(f"Total Ally Jungle Minions Killed: {totalAllyJungleMinionsKilled}")
    print(f"Total Enemy Jungle Minions Killed: {totalEnemyJungleMinionsKilled}\n")

takedowns = getTakedowns(postData, targetPuuid)
print(f"TakeDowns by {targetChampionName}: {takedowns}")

soloKills = getSoloKills(postData, targetPuuid)
print(f"Solo Kills for {targetChampionName}: {soloKills}")

kda = getKda(postData, targetPuuid)
print(f"KDA by {targetChampionName}: {kda}")

killParticipation = getKillParticipation(postData, targetPuuid)
print(f"Kill Participation for {targetChampionName}: {killParticipation}\n")


totalDamageDealtToChampions = getTotalDamageDealtToChampions(postData, targetPuuid)
print(f"Total Damage Dealt To Champions by {targetChampionName}: {totalDamageDealtToChampions}")

totalDamageTaken = getTotalDamageTaken(postData, targetPuuid)
print(f"Total Damage Taken for {targetChampionName}: {totalDamageTaken}")

deathsByEnemyChamps = getDeathsByEnemyChamps(postData, targetPuuid)
print(f"Deaths by Enemy Champions on {targetChampionName}: {deathsByEnemyChamps}\n")


visionScore = getVisionScore(postData, targetPuuid)
print(f"Vison Score for {targetChampionName}: {visionScore}")
    
visionScoreAdvantageLaneOpponent = getVisionScoreAdvantageLaneOpponent(postData, targetPuuid)
print(f"Vision Score Advantage on Lane Opponent for {targetChampionName}: {visionScoreAdvantageLaneOpponent}")

wardsPlaced = getWardsPlaced(postData, targetPuuid)
print(f"Wards Placed by {targetChampionName}: {wardsPlaced}")

controlWardTimeCoverageInRiverOrEnemyHalf = getControlWardTimeCoverageInRiverOrEnemyHalf(postData, targetPuuid)
print(f"Control Ward Time Coverage In River or Enemy Half for {targetChampionName}: {controlWardTimeCoverageInRiverOrEnemyHalf}")

controlWardsPlaced = getControlWardsPlaced(postData, targetPuuid)
print(f"Control Wards placed by {targetChampionName}: {controlWardsPlaced}")

stealthWardsPlaced = getStealthWardsPlaced(postData, targetPuuid)
print(f"Stealth Wards Placed by {targetChampionName}: {stealthWardsPlaced}")

wardsKilled = getWardsKilled(postData, targetPuuid)
print(f"Wards Killed by {targetChampionName}: {wardsKilled}\n")


teamBaronKills = getTeamBaronKills(postData, targetPuuid)
print(f"Team Baron Kills by {targetChampionName}: {teamBaronKills}")

teamRiftHeraldKills = getTeamRiftHeraldKills(postData, targetPuuid)
print(f"Team Rift Herald Kills by {targetChampionName}: {teamRiftHeraldKills}")

teamElderDragonKills = getTeamElderDragonKills(postData, targetPuuid)
print(f"Team Elder Dragon Kills by {targetChampionName}: {teamElderDragonKills}")

dragonKills = getDragonKills(postData, targetPuuid)
print(f"Dragon Kills by {targetChampionName}: {dragonKills}")

dragonTakedowns = getDragonTakedowns(postData, targetPuuid)
print(f"Dragon Takedowns by {targetChampionName}: {dragonTakedowns}")

turretTakedowns = getTurretTakedowns(postData, targetPuuid)
print(f"Turret Takedowns by {targetChampionName}: {turretTakedowns}\n")


goldPerMinute = getGoldPerMinute(postData, targetPuuid)
print(f"Gold Per Minute for {targetChampionName}: {goldPerMinute}")

laneMinionsFirst10Minutes = getLaneMinionsFirst10Minutes(postData, targetPuuid)
if laneRole != "JUNGLE":
    print(f"Lane Minions First 10 Minutes by {targetChampionName}: {laneMinionsFirst10Minutes}\n")
    
teamSide = getTeamSide(postData, targetPuuid)
if teamSide == 100:
    print(f"{targetChampionName} was Blue Side")
else:
    print(f"{targetChampionName} was Red Side")

gameWon = getGameWon(postData, targetPuuid)
if gameWon == True:
    print(f"{targetChampionName}'s Team Won the Game")
else:
    print(f"{targetChampionName}'s Team Lost the Game")

Role of MasterYi: JUNGLE
Total Ally Jungle Minions Killed: 92
Total Enemy Jungle Minions Killed: 16

TakeDowns by MasterYi: 34
Solo Kills for MasterYi: 5
KDA by MasterYi: 2.8333333333333335
Kill Participation for MasterYi: 0.68

Total Damage Dealt To Champions by MasterYi: 51385
Total Damage Taken for MasterYi: 49204
Deaths by Enemy Champions on MasterYi: 12

Vison Score for MasterYi: 38
Vision Score Advantage on Lane Opponent for MasterYi: 0.44010066986083984
Wards Placed by MasterYi: 9
Control Ward Time Coverage In River or Enemy Half for MasterYi: 0.44759596864850204
Control Wards placed by MasterYi: 2
Stealth Wards Placed by MasterYi: 7
Wards Killed by MasterYi: 2

Team Baron Kills by MasterYi: 1
Team Rift Herald Kills by MasterYi: 0
Team Elder Dragon Kills by MasterYi: 1
Dragon Kills by MasterYi: 2
Dragon Takedowns by MasterYi: 2
Turret Takedowns by MasterYi: 1

Gold Per Minute for MasterYi: 498.52359737296433
MasterYi was Blue Side
MasterYi's Team Won the Game


In [17]:
def getPlayerStatsAndExtra(data, targetPuuid, participantData):
    for participant in participantData:
        participantId = participant[0]
        participantPuuid = participant[1]

        # Determine the extra participantId based on the given participantId
        extraParticipantId = str((int(targetParticipantId) + 5) % 10) if int(targetParticipantId) != 5 else '10'

        print(f"Target Participant ID: {targetParticipantId}, PUUID: {targetPuuid}")
        print(f"Extra Participant ID: {extraParticipantId}")

        # Get stats for the target participant
       
        targetStats = {
            'Player': f"{targetChampionName}",
            'visionScoreAdvantageLaneOpponent': getVisionScoreAdvantageLaneOpponent(data, targetPuuid),
            'visionScore': getVisionScore(data, targetPuuid),
            'killParticipation': getKillParticipation(data, targetPuuid),
            # Add more stats using your specific functions
        }
            

        # Find the correct extra participant from participantData
        extraParticipant = next((p for p in participantData if p[0] == int(extraParticipantId)), None)
        if extraParticipant:
            extraParticipantPuuid = extraParticipant[1]
            # Assign extraParticipantName inside the loop
            extraParticipantName = extraParticipant[2]
            # Get stats for the extra participant
            extraStats = {
                'Enemy Player': extraParticipantName,
                'visionScoreAdvantageLaneOpponent': getVisionScoreAdvantageLaneOpponent(data, extraParticipantPuuid),
                'visionScore': getVisionScore(data, extraParticipantPuuid),
                'killParticipation': getKillParticipation(data, extraParticipantPuuid),
                # Add more stats using your specific functions
            }

            return targetStats, extraStats

    return None, None

# Example usage:
targetStats, extraStats = getPlayerStatsAndExtra(postData, targetPuuid, participantData)

if targetStats is not None and extraStats is not None:
    print("Target Participant Stats:")
    print(targetStats)

    print("\nExtra Participant Stats:")
    print(extraStats)
else:
    print(f"No participant found with PUUID: {targetPuuid}")


Target Participant ID: 2, PUUID: 7hSyI8OKEFPpadVyhqKmTOJlT6umda2FMt1sAeIcZ1WcxdATw7Tvu4ixaR8llqv9u2qPutnAsL9Shg
Extra Participant ID: 7
Target Participant Stats:
{'Player': 'MasterYi', 'visionScoreAdvantageLaneOpponent': 0.44010066986083984, 'visionScore': 38, 'killParticipation': 0.68}

Extra Participant Stats:
{'Enemy Player': 'Khazix', 'visionScoreAdvantageLaneOpponent': -0.30560410022735596, 'visionScore': 26, 'killParticipation': 0.6153846153846154}


In [18]:
def printRunesForTargetPuuid(postData, runeSheet, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            perks = participant.get('perks', {})
            
            # Iterate through styles array in perks
            for style in perks.get('styles', []):
                primaryStyleId = style.get('style', 0)
                print(f"PrimaryStyle Rune ID: {primaryStyleId}, Type: {type(primaryStyleId)}")

                # Iterate through selections array in primaryStyle
                for selection in style.get('selections', []):
                    lesserRuneId = selection.get('perk', 0)
                    print(f"Lesser Rune ID: {lesserRuneId}")

                    # Check if the lesser rune ID is present in runeSheet
                    matching_runes = [rune for rune in runeSheet if rune.get('id') == lesserRuneId]
                    if matching_runes:
                        matching_rune = matching_runes[0]
                        print(f"Lesser Rune ID: {lesserRuneId}, Name: {matching_rune.get('name')}")
                    else:
                        print(f"No matching rune found for ID: {lesserRuneId}")

printRunesForTargetPuuid(postData, runeSheet, targetPuuid)


PrimaryStyle Rune ID: 8000, Type: <class 'int'>
Lesser Rune ID: 8008
No matching rune found for ID: 8008
Lesser Rune ID: 9111
No matching rune found for ID: 9111
Lesser Rune ID: 9104
No matching rune found for ID: 9104
Lesser Rune ID: 8014
No matching rune found for ID: 8014
PrimaryStyle Rune ID: 8100, Type: <class 'int'>
Lesser Rune ID: 8138
No matching rune found for ID: 8138
Lesser Rune ID: 8105
No matching rune found for ID: 8105
